In [1]:
!nvidia-smi

Thu Apr 14 03:16:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [4]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c fake-news

 71% 33.0M/46.5M [00:00<00:00, 92.5MB/s]
100% 46.5M/46.5M [00:00<00:00, 116MB/s] 


In [6]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [1]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
from sklearn.metrics import f1_score
from transformers import AutoConfig

In [2]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"

In [3]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'

In [4]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(fake_news_dataset_path, exist_ok=True)

# Load fake news dataset from kaggle

In [13]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.drop(['id','title','author'],axis=1)
df=df.sample(frac=1)
df

,text,label
6740,(7 fans) - Advertisement - Amid mounting anger...,1
5972,Facebook reportedly pays an army of Filipino c...,0
7751,Our New Country: Women And Minorities Hit Hard...,1
20440,A team of cartel gunmen began to fire indiscri...,0
8656,"\nIn a truly shocking twist, the Supreme Court...",1
...,...,...
14283,"November 2, 2016 555 Russian and Turkish milit...",1
11050,"In 1995, furious over quality problems with on...",0
14926,Braunschweiger Criminal Police chief Ulf Küche...,0
1339,While ordinary Americans nationwide work on Th...,0


Split raw dataset to train set and validation set

In [14]:
train=df.iloc[:-5200]
valid=df.iloc[-5200:]

In [15]:
len(train),len(valid)

(15600, 5200)

Check data whether it is balanced or not

In [16]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 7761 samples
label: 1	has 7839 samples


In [17]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 2626 samples
label: 1	has 2574 samples


In [25]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [26]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [27]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'text'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'text'])


In [28]:
"""This function is from The Stanford Question Answering Dataset evaluate"""
def normalize(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  return white_space_fix(remove_articles(remove_punc(s))).strip()

In [29]:
train['text']=train['text'].map(normalize)

In [30]:
valid['text']=valid['text'].map(normalize)

In [31]:
train=train[train['text']!=""]
valid=valid[valid['text']!=""]

In [32]:
len(train),len(valid)

(15513, 5171)

In [33]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [5]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(name)

In [7]:
max_length=512
return_tensors='tf'

In [8]:
x_train=tokenizer(train['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [9]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [10]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [11]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [12]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [13]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [14]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [15]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [16]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

In [27]:
epochs=10
num_train_steps =epochs*num_training_batches
num_warmup_steps=5000
lr=1e-5

Load the pretrained model

In [28]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
optimizer,_=create_optimizer(init_lr =lr,num_train_steps=num_train_steps,num_warmup_steps=num_warmup_steps)

In [30]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Define train_step function for training

In [31]:
train_step_signature= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature)
def train_step(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = True)
    loss=loss_fn(labels,outputs.logits)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

Defind test_step function for validation

In [32]:
def test_step(input_ids, attention_mask,labels):
  
  outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = False)
  loss=loss_fn(labels,outputs.logits)

  return loss

In [33]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Initializing the new checkpoint


In [34]:
threshold=1e-5

Train

In [35]:
def run_step(dataset,session='train'):
  
  step=train_step if session=='train' else test_step

  loop = tqdm(dataset)
  loss=0

  num_batches=0
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']

    _loss=step(input_ids=input_ids,attention_mask=attention_mask,labels=labels)


    _loss=_loss.numpy()
    loss+=_loss
    num_batches+=1
    loop.set_postfix({'train_loss_per_batch':_loss})
    loop.update()


  loss=loss/num_batches
  return loss

In [ ]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_loss=run_step(train_dataset)

  """Validation"""
  valid_loss=run_step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  ckpt_save_path = ckpt_manager.save()
  # if tf.abs(min_loss-valid_loss)>threshold and min_loss>valid_loss:
  #   ckpt_save_path = ckpt_manager.save()
  #   print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')
  #   min_loss=valid_loss

  0%|          | 0/1938 [00:00<?, ?it/s]

# Inferencing

Load test set

In [ ]:
test=pd.read_csv("/content/test.csv")

In [ ]:
submit=pd.read_csv("/content/submit.csv")
labels=tf.convert_to_tensor(submit['label'],dtype=tf.float32)

In [ ]:
test['text']=test['text'].astype(str).map(normalize)

In [ ]:
samples=test['text'].tolist()

In [ ]:
num_labels=len(submit['label'].unique())
num_labels

2

Load model

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [ ]:
model=TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.num_labels

2

In [ ]:
max_length=512
return_tensors='tf'
tokenizer = AutoTokenizer.from_pretrained(name)

Load the weights

In [ ]:
ckpt_path = model_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

Loading the latest checkpoint from /content/drive/MyDrive/fake_news/models/bert-base-uncased/ckpt-1


In [ ]:
y_pred=[]
loop = tqdm(range(len(samples)))
for idx,sample in enumerate(samples):
  inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)
  outputs=model(**inputs,training=False)
  prob=tf.nn.softmax(outputs.logits,axis=1)
  preds=tf.argmax(prob,axis=1)
  y_pred.append(preds)
  loop.update()


100%|██████████| 5200/5200 [15:35<00:00,  5.74it/s]

In [ ]:
y_pred=tf.convert_to_tensor(y_pred)

In [ ]:
y_pred=tf.reshape(y_pred,shape=(y_pred.shape[0],))
y_pred

<tf.Tensor: shape=(5200,), dtype=int64, numpy=array([0, 1, 1, ..., 0, 1, 0])>

In [ ]:
labels

<tf.Tensor: shape=(5200,), dtype=float32, numpy=array([0., 1., 0., ..., 0., 1., 0.], dtype=float32)>

In [ ]:
f1_score(labels,y_pred,average='micro')